In [6]:
import sys
sys.path.append('../python_scripts')
import pymesh
import numpy as np
import logging, sys
logging.disable(sys.maxsize)
dirname = "/home/jxt/PPPM-TDBEM/dataset/bowl"
from fem import FEMmodel, Material, MatSet, LOBPCG_solver, SoundObj
from visulize import viewer
obj = SoundObj(dirname)

original mesh data
vertices:  (18370, 3)
faces:  (36736, 3)
bbox_min:  [-0.07172556 -0.03272797 -0.07172596]
bbox_max:  [0.07172556 0.02840117 0.0717259 ]
target cell_size:  0.0047817284295000005


In [7]:
obj.tetrahedralize()
obj.modal_analysis(k=200, material=Material(MatSet.Plastic))

tetrahedralized mesh data
vertices:  (3943, 3)
faces:  (7262, 3)
voxels:  (12190, 4)


In [8]:
obj.show_frequencys(num=20)
obj.visualize_modes(viewer, num=200)

[ 623.64056  629.6274  1307.8104  1311.197   2177.2021  2177.6624
 2401.4448  2587.7415  2591.974   3094.4365  3198.25    3230.5005
 3534.5369  3537.3916  3818.568   3823.0872  4044.6372  4054.9875
 4236.886   4239.6167 ]


Box(children=(Box(children=(IntSlider(value=0, description='Mode Index', layout=Layout(flex='3 1 auto'), max=1…

In [9]:
vertices = obj.tet_mesh.vertices
tets = obj.tet_mesh.voxels
print(vertices.shape)
print(tets.shape)
print(obj.eigenvalues.shape)
print(obj.eigenvectors.shape)
np.savetxt(dirname + "/vertices.txt", vertices)
np.savetxt(dirname + "/tets.txt", tets.astype(int))
np.savetxt(dirname + "/eigenvalues.txt", obj.eigenvalues)
np.savetxt(dirname + "/eigenvectors.txt", obj.eigenvectors)

(3943, 3)
(12190, 4)
(200,)
(11829, 200)


In [10]:
from physical import PhysicalAnimation
import pybullet as p
anim = PhysicalAnimation(dirname)
anim.generate()
anim.post_process_contacts()

Generating animation...


  1%|          | 168/30000 [00:00<00:28, 1059.01it/s]

ven = NVIDIA Corporation
ven = NVIDIA Corporation


100%|██████████| 30000/30000 [00:05<00:00, 5891.68it/s]


post processing contact forces....


100%|██████████| 30000/30000 [00:02<00:00, 14885.93it/s]


In [11]:
SR = 44100
from tqdm import tqdm
import scipy.signal
from audio import save_audio, show_audio, IIR
sound_signal = np.zeros(int(anim.time_length * SR))
f = np.zeros((len(obj.eigenvalues), len(sound_signal)))
for contact in tqdm(anim.contact):
    t = int(contact[0] * SR)
    vertex_id = int(contact[1])
    force_direction = contact[2:5]
    impulse = contact[5] * (obj.eigenvectors[vertex_id*3, :] * force_direction[0] +
                             obj.eigenvectors[vertex_id*3+1, :] * force_direction[1] +
                             obj.eigenvectors[vertex_id*3+2, :] * force_direction[2])
    f[:, t] += impulse
# smooth force 
filter_width = 21
coeffs = np.sin(np.linspace(0, np.pi, filter_width))
coeffs /= np.sum(coeffs)
filter = np.zeros((len(obj.eigenvalues), len(sound_signal)))
for mode_idx in tqdm(range(len(obj.eigenvalues))):
    filter[mode_idx, :] = scipy.signal.convolve(f[mode_idx, :], coeffs, mode='same')
f = filter

for mode_idx in range(200):
    sound_signal += IIR(f[mode_idx], obj.eigenvalues[mode_idx], obj.fem_model.material.alpha, obj.fem_model.material.beta, 1/SR)

# get acceleration
sound_signal = np.diff(sound_signal, n=2)
save_audio(sound_signal, SR, dirname + "/sound.wav")

100%|██████████| 200/200 [00:00<00:00, 353.48it/s]


In [12]:
import subprocess
from IPython.display import Video
video_name = dirname + '/animation.mp4'
audio_name = dirname + '/sound.wav'
output_name = dirname + '/animation+sound.mp4'
subprocess.run(['ffmpeg', '-hide_banner', '-loglevel', 'error', '-i', video_name, '-i', audio_name, '-c:v', 'copy', '-c:a', 'libvorbis', '-f', 'mp4', output_name, '-y'])
Video(output_name)